I need 4 different notebooks for:
1. Building baseline models with classifier (DONE) then ensemble
2. Building baseline models with regressors then ensemble
3. Building CNNS models with classifiers then ensemble
4. Building CNNS models with regressors then ensemble

In [ ]:
import os
import joblib
print(joblib.__version__)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

1.3.2


In [ ]:
def bin_ages(age):
    if pd.isnull(age) or not isinstance(age, (str, float, int)):
        return 'Unknown'
    age_str = str(age)

    if 'pcw' in age_str:
        weeks = int(age_str.split(' ')[0])
        if weeks >= 4 and weeks <= 7: # 4-7 pcw
            return 'Embryonic'
        elif weeks >= 8 and weeks <= 38: # 8-38 pcw
            return 'Prenatal'
    elif 'mos' in age_str or 'M' in age_str:
        months = int(age_str.split(' ')[0])
        if months >= 0 and months <= 19: # 0-19 months
            return 'Infancy'
    elif 'yrs' in age_str or 'Y' in age_str:
        years = int(age_str.split(' ')[0])
        if years >= 1 and years <= 11: # 1-11 years
            return 'Childhood'
        elif years >= 12 and years <= 19: # 12-19 years
            return 'Adolescence'
        elif years >= 20: # 20 years and above
            return 'Adulthood'

    # return 'Unknown' for any other cases
    return 'Unknown'

In [ ]:
def scale(X):
    """
    Scales (standardizes) the input data.

    Args:
    - X (pd.DataFrame): Input data to be scaled.

    Returns:
    - np.ndarray: Scaled (standardized) data.
    """
    scaler = StandardScaler()
    return scaler.fit_transform(X)

In [ ]:
def aggregate_data(data):
    """
    Aggregate the data by taking the mean across all brain regions for each donor.

    Args:
    - data (DataFrame): The input RNAseq data, with donor_brainRegion identifiers as the index.

    Returns:
    - DataFrame: The aggregated data.
    """

    data['donor_id_mod'] = data.index.to_series().apply(lambda x: x.rsplit('_', 1)[0])

    age_data = data[['donor_id_mod', 'age_group']].drop_duplicates()

    data = data.drop(columns=['age_group'], errors='ignore')

    aggregated_data = data.groupby('donor_id_mod').mean()

    aggregated_data = pd.merge(aggregated_data, age_data, left_index=True, right_on='donor_id_mod', how='left').set_index('donor_id_mod')

    print("DATA IS AGGREGATED", aggregated_data)
    return aggregated_data


In [ ]:
def preprocess_chunk(chunk, data_type, subset_rows=20):
    """
    Preprocesses a chunk of data. It subsets the chunk, bins ages, maps labels,
    filters relevant columns based on the data type, and returns the features and labels.

    Args:
    - chunk (pd.DataFrame): Input data chunk to be preprocessed.
    - data_type (str): Type of data ('Methylation' currently).
    - subset_rows (int, optional): Number of rows to retain from the top of the chunk. Defaults to 20.

    Returns:
    - X (pd.DataFrame): Processed feature data.
    - y (pd.Series): Processed labels.

    Raises:
    - ValueError: If an unsupported data_type is provided.
    """
    chunk = chunk.head(subset_rows)

    if 'Age' not in chunk.columns:
        print("Warning: Age column not found in this chunk. Skipping.")
        return pd.DataFrame(), pd.Series()

    chunk['age_group'] = chunk['Age'].apply(bin_ages)

    label_map = {'Embryonic': 0, 'Prenatal': 1, 'Infancy': 2, 'Childhood': 3, 'Adolescence': 4, 'Adulthood': 5}
    chunk['age_group'] = chunk['age_group'].map(label_map)

    if data_type == 'Methylation':
        relevant_columns = [col for col in chunk.columns if col.startswith(('cg', 'rs', 'ch'))] + ['Age', 'age_group']
        filtered_chunk = chunk[relevant_columns]
        X = filtered_chunk.drop(['Age', 'age_group'], axis=1)
        y = filtered_chunk['age_group']
    else:
        raise ValueError("Unsupported data type for chunk processing")

    if y.isnull().any():
        print("NaNs present in age_group labels!")
        print(chunk[chunk['age_group'].isnull()]['Age'])

    return X, y

In [ ]:
def preprocess_data(data_path, file_type='csv'):
    """
    Preprocess the data and return the train-test split.

    Args:
    - data_path (str): Path to the data file.
    - file_type (str): File format ('csv', 'excel', 'txt').

    Returns:
    - X_train, X_test, y_train, y_test: Train-test split of the preprocessed data.
    """
    subset_rows = 20


    if 'methylation' in data_path.lower() and file_type == 'csv':
        chunk_size = 5
        chunks = pd.read_csv(data_path, chunksize=chunk_size)

        X_list = []
        y_list = []
        for chunk in chunks:
            chunk = chunk.head(subset_rows)
            X_chunk, y_chunk = preprocess_chunk(chunk, 'Methylation')

            X_list.append(X_chunk)
            y_list.append(y_chunk)

        X_data = pd.concat(X_list, axis=0)
        y_data = pd.concat(y_list, axis=0)

        data = pd.concat([X_data, y_data], axis=1)
        print("Columns after processing all chunks:", data.columns)

    else:
        if file_type == 'csv':
            data = pd.read_csv(data_path, index_col=0)
        elif file_type == 'excel':
            data = pd.read_excel(data_path, index_col=0)
        elif file_type == 'txt':
            data = pd.read_csv(data_path, sep='\t', index_col=0)
        else:
            raise ValueError("Unsupported file type")
        print("Columns after determining data type:", data.columns)

    if 'rnaseq' in data_path.lower():
        data_type = 'RNA-Seq'
    elif 'methylation' in data_path.lower():
        data_type = 'Methylation'
    elif 'microrna' in data_path.lower():
        data_type = 'MicroRNA'
    else:
        raise ValueError("Unknown data type")

    print(data.columns)

    data['age_group'] = data['age'].apply(bin_ages)

    if data_type == 'RNA-Seq':
        label_map = {'Prenatal': 0, 'Infancy': 0, 'Childhood': 1, 'Adolescence': 2, 'Adulthood': 3}
    elif data_type == 'MicroRNA':
        label_map = {'Infancy': 0, 'Childhood': 1, 'Adolescence': 2, 'Adulthood': 3}
    else:
        label_map = {'Embryonic': 0, 'Prenatal': 1, 'Infancy': 2, 'Childhood': 3, 'Adolescence': 4, 'Adulthood': 5}

    data['age_group'] = data['age_group'].map(label_map)

    if data_type == 'RNA-Seq':
        data_numeric = data.drop(['age', 'age_group'], axis=1)
        one_percent_of_samples = data_numeric.shape[1] * 0.01
        mask = data_numeric.gt(1).sum(axis=1) >= one_percent_of_samples
        filtered_data = data[mask]
        X = filtered_data.drop(['age', 'age_group'], axis=1)
        y = filtered_data['age_group']

    elif data_type == 'Methylation':
        relevant_columns = [col for col in data.columns if col.startswith(('cg', 'rs', 'ch'))] + ['age_group']
        filtered_data = data[relevant_columns]
        X = filtered_data.drop(['age_group'], axis=1)
        y = filtered_data['age_group']

    elif data_type == 'MicroRNA':
        relevant_columns = [col for col in data.columns if col.startswith('hsa')] + ['age', 'age_group']
        filtered_data = data[relevant_columns]
        X = filtered_data.drop(['age', 'age_group'], axis=1)
        y = filtered_data['age_group']

    if y.isnull().any():
        print("NaNs present in age_group labels!")
        print(data[data['age_group'].isnull()]['age'])

    print(np.unique(y))
    print("About to shuffle")

    X, y = shuffle(X, y, random_state=0)
    X_scaled = scale(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.2)

    print("The data is split")
    return X_train, X_test, y_train, y_test

In [ ]:
def save_data_splits(X_train, X_test, y_train, y_test, output_dir):
    """
    Saves the train-test data splits to the specified directory using joblib.

    Args:
    - X_train (pd.DataFrame or np.ndarray): Training data features.
    - X_test (pd.DataFrame or np.ndarray): Testing data features.
    - y_train (pd.Series or np.ndarray): Training data labels.
    - y_test (pd.Series or np.ndarray): Testing data labels.
    - output_dir (str): Directory path where the data splits will be saved.

    Note:
    - If the output directory does not exist, it will be created.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    joblib.dump(X_train, os.path.join(output_dir, 'X_train_classifiers.pkl'))
    joblib.dump(X_test, os.path.join(output_dir, 'X_test_classifiers.pkl'))
    joblib.dump(y_train, os.path.join(output_dir, 'y_train_classifiers.pkl'))
    joblib.dump(y_test, os.path.join(output_dir, 'y_test_classifiers.pkl'))

In [ ]:
def train_evaluate_classifier(classifier, X_train, y_train, X_test, y_test, output_dir):
    """
    Trains the classifier on the training data, evaluates it on the test data,
    and saves the trained model and the classification report to the specified directory.

    Args:
    - classifier (estimator): An instance of a classifier that follows the scikit-learn API (has `fit` and `predict` methods).
    - X_train (pd.DataFrame or np.ndarray): Training data features.
    - y_train (pd.Series or np.ndarray): Training data labels.
    - X_test (pd.DataFrame or np.ndarray): Testing data features.
    - y_test (pd.Series or np.ndarray): Testing data labels.
    - output_dir (str): Directory path where the trained model and the classification report will be saved.

    Returns:
    - y_pred (np.ndarray): Predicted labels for the test data.
    - classifier (estimator): The trained classifier.

    Note:
    - If any class in the test set does not have a predicted sample,
      a warning is raised and the precision, recall, and F-beta scores are set to 0.0.
    """
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    report = classification_report(y_test, y_pred, zero_division=1)

    print("Classification Report:")
    print(report)

    with open(os.path.join(output_dir, 'classification_report.txt'), 'w') as f:
        f.write(report)

    joblib.dump(classifier, os.path.join(output_dir, 'trained_classifier.pkl'))

    return y_pred, classifier

In [ ]:
def plot_roc_curve(y_test, y_prob, output_dir):
    """
    Plots the Receiver Operating Characteristic (ROC) curve for multi-class classification and saves the plot.

    Args:
    - y_test (pd.Series or np.ndarray): True labels for the test data.
    - y_prob (np.ndarray): Probability estimates of the positive class for each class.
    - output_dir (str): Directory path where the ROC curve plot will be saved.

    Note:
    - The function handles multi-class classification by plotting an ROC curve for each class.
    - If the provided `output_dir` does not exist, it must be created before calling this function.
    """
    n_classes = len(np.unique(y_test))
    mlb = MultiLabelBinarizer(classes=list(range(n_classes)))
    y_test_binarized = mlb.fit_transform(y_test.to_numpy().reshape(-1, 1))

    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    macro_roc_auc = roc_auc_score(y_test_binarized, y_prob, average='macro')
    for i in range(n_classes):
        print(f"Class {i} ROC AUC: {roc_auc[i]:.2f}")
    print(f"Macro-average ROC AUC: {macro_roc_auc:.2f}")

    colors = ['blue', 'red', 'green', 'orange', 'purple']
    plt.figure(figsize=(10, 6))
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'ROC curve (area = {roc_auc[i]:.2f}) for class {i}')
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.savefig(os.path.join(output_dir, 'roc_curve.png'))
    plt.close()


In [ ]:
def main(data_paths):
    """
    The main pipeline to preprocess data, train multiple classifiers, evaluate their performance,
    and plot the ROC curves for multi-class classification.

    Args:
    - data_paths (list of str): A list of paths to the datasets.

    Workflow:
    1. For each data path, identify the data type from its name.
    2. Preprocess the data and save the train-test splits.
    3. For each classifier, train it using the training data and evaluate on the test data.
    4. For each trained classifier, predict the class probabilities on the test data.
    5. Plot the ROC curve and save it.

    Note:
    - The results, including the train-test splits, trained models, classification reports, and ROC plots,
      are saved in subdirectories organized by data type and classifier name under 'baseline_model_outputs'.
    """
    classifiers = [
        OneVsRestClassifier(XGBClassifier()),
        OneVsRestClassifier(SVC(probability=True, random_state=42)),
        OneVsRestClassifier(MLPClassifier())
    ]

    for data_path in data_paths:

        data_type = os.path.basename(data_path).split('_')[0]

        X_train, X_test, y_train, y_test = preprocess_data(data_path)
        save_data_splits(X_train, X_test, y_train, y_test, os.path.join('baseline_classifier_outputs', data_type))

        for classifier in classifiers:
            underlying_classifier_name = classifier.estimator.__class__.__name__

            output_dir = os.path.join('baseline_classifier_outputs', data_type, underlying_classifier_name)
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            print(f"Training classifier: OneVsRest with {underlying_classifier_name}")

            y_pred, trained_classifier = train_evaluate_classifier(classifier, X_train, y_train, X_test, y_test, output_dir)
            y_prob = trained_classifier.predict_proba(X_test)
            plot_roc_curve(y_test, y_prob, output_dir)

In [ ]:
data_paths = [#'methylation_1.csv',
              'microRNA_1.csv',
              'rnaseq_1.csv'
              ]
main(data_paths)

Columns after determining data type: Index(['hsa-miR-26a-5p', 'hsa-miR-181a-5p', 'hsa-miR-143-3p', 'hsa-let-7a-5p',
       'hsa-miR-9-5p', 'hsa-miR-3182', 'hsa-miR-99b-5p', 'hsa-miR-30a-5p',
       'hsa-miR-27b-3p', 'hsa-miR-191-5p',
       ...
       'hsa-miR-4653-5p', 'hsa-miR-4264', 'hsa-miR-3119', 'hsa-miR-4330',
       'hsa-miR-4318', 'hsa-miR-4279', 'hsa-miR-3689f', 'hsa-miR-4291',
       'donor_name', 'age'],
      dtype='object', length=1863)
Index(['hsa-miR-26a-5p', 'hsa-miR-181a-5p', 'hsa-miR-143-3p', 'hsa-let-7a-5p',
       'hsa-miR-9-5p', 'hsa-miR-3182', 'hsa-miR-99b-5p', 'hsa-miR-30a-5p',
       'hsa-miR-27b-3p', 'hsa-miR-191-5p',
       ...
       'hsa-miR-4653-5p', 'hsa-miR-4264', 'hsa-miR-3119', 'hsa-miR-4330',
       'hsa-miR-4318', 'hsa-miR-4279', 'hsa-miR-3689f', 'hsa-miR-4291',
       'donor_name', 'age'],
      dtype='object', length=1863)
[0 1 2 3]
About to shuffle
The data is split
Training classifier: OneVsRest with XGBClassifier
Accuracy: 84.09%
Classification 

In [ ]:
!zip -r baseline_model_outputs.zip baseline_model_outputs2

  adding: baseline_model_outputs2/ (stored 0%)
  adding: baseline_model_outputs2/rnaseq/ (stored 0%)
  adding: baseline_model_outputs2/rnaseq/SVC/ (stored 0%)
  adding: baseline_model_outputs2/rnaseq/SVC/roc_curve.png (deflated 15%)
  adding: baseline_model_outputs2/rnaseq/SVC/trained_model.pkl (deflated 6%)
  adding: baseline_model_outputs2/rnaseq/SVC/classification_report.txt (deflated 64%)
  adding: baseline_model_outputs2/rnaseq/y_train.pkl (deflated 87%)
  adding: baseline_model_outputs2/rnaseq/X_test.pkl (deflated 6%)
  adding: baseline_model_outputs2/rnaseq/MLPClassifier/ (stored 0%)
  adding: baseline_model_outputs2/rnaseq/MLPClassifier/roc_curve.png (deflated 15%)
  adding: baseline_model_outputs2/rnaseq/MLPClassifier/trained_model.pkl (deflated 5%)
  adding: baseline_model_outputs2/rnaseq/MLPClassifier/classification_report.txt (deflated 66%)
  adding: baseline_model_outputs2/rnaseq/X_train.pkl (deflated 6%)
  adding: baseline_model_outputs2/rnaseq/XGBClassifier/ (stored 0%)


In [ ]:
from google.colab import files
files.download('baseline_model_outputs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>